## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-30-20-52-09 +0000,nyt,U.N. Security Council Approves Larger Security...,https://www.nytimes.com/2025/09/30/world/ameri...
1,2025-09-30-20-50-32 +0000,nyt,"Earthquake in Philippines Kills at Least 20, O...",https://www.nytimes.com/2025/09/30/world/asia/...
2,2025-09-30-20-49-42 +0000,nyt,Crash Sends Kegs and Cans Onto California Free...,https://www.nytimes.com/2025/09/30/us/beer-keg...
3,2025-09-30-20-45-57 +0000,cbc,Nicole Kidman files for divorce from Keith Urb...,https://www.cbc.ca/news/entertainment/nicole-k...
4,2025-09-30-20-45-01 +0000,nyt,Trump Says a Deal Is Close With Harvard,https://www.nytimes.com/2025/09/30/us/trump-ha...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2347/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
28,trump,67
43,shutdown,25
29,deal,20
40,government,19
291,gaza,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
264,2025-09-30-03-05-00 +0000,wsj,Top Democrats made their case to Trump for res...,https://www.wsj.com/politics/policy/government...,170
333,2025-09-29-21-14-29 +0000,nypost,Trump unveils 20-point Gaza cease-fire plan wi...,https://nypost.com/2025/09/29/us-news/trump-un...,151
148,2025-09-30-15-10-46 +0000,nypost,Trump gives Hamas ‘3 or 4 days’ to accept his ...,https://nypost.com/2025/09/30/world-news/trump...,136
15,2025-09-30-20-27-00 +0000,wsj,President Trump hailed the use of military for...,https://www.wsj.com/politics/national-security...,134
249,2025-09-30-06-50-20 +0000,nypost,Trump mocks Chuck Schumer and Hakeem Jeffries ...,https://nypost.com/2025/09/30/us-news/trump-mo...,133


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
264,170,2025-09-30-03-05-00 +0000,wsj,Top Democrats made their case to Trump for res...,https://www.wsj.com/politics/policy/government...
333,84,2025-09-29-21-14-29 +0000,nypost,Trump unveils 20-point Gaza cease-fire plan wi...,https://nypost.com/2025/09/29/us-news/trump-un...
15,57,2025-09-30-20-27-00 +0000,wsj,President Trump hailed the use of military for...,https://www.wsj.com/politics/national-security...
101,49,2025-09-30-17-01-00 +0000,wsj,American Eagle Outfitters’ chief executive had...,https://www.wsj.com/business/retail/jay-schott...
109,45,2025-09-30-16-30-00 +0000,wsj,Congress’s decision about what to do about exp...,https://www.wsj.com/politics/policy/government...
23,39,2025-09-30-20-06-32 +0000,wapo,Louisiana governor requests National Guard for...,https://www.washingtonpost.com/nation/2025/09/...
35,35,2025-09-30-19-41-34 +0000,nypost,Inside NYC hoops star’s fall from on-court her...,https://nypost.com/2025/09/30/us-news/inside-n...
118,35,2025-09-30-16-13-32 +0000,latimes,Texas firm proposes major change in California...,https://www.latimes.com/california/story/2025-...
178,33,2025-09-30-13-33-28 +0000,nypost,Wild video shows drunk passenger being yanked ...,https://nypost.com/2025/09/30/world-news/drunk...
85,29,2025-09-30-17-35-27 +0000,wapo,Trump administration illegally targeted pro-Pa...,https://www.washingtonpost.com/education/2025/...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
